In [120]:
import numpy as np
import json

In [121]:
def load_from_file(key):
    res = []
    # append runs from ycsb
    if key in ['mongo', 'redis', 'postgres', 'cassandra', 'memcache']:
        for workload in ['a', 'b', 'f']:
            res.append(list(json.load(open('../results/ycsb/{}_{}.json'.format(key,workload), 'r')).keys()))

    # append runs from cloudSuite webServing
    if key in ['elgg', 'memcache', 'mariadb']:
        for workload in ['', '_run', '_manual']:
            res.append(list(json.load(open('../results/cloudsuite/webServing_{}{}.json'.format(key,workload), 'r')).keys()))

    # append runs from cloudSuite dataServing
    if key == 'cassandra':
        for workload in ['', '_load']:
            res.append(list(json.load(open('../results/cloudsuite/dataServing{}.json'.format(workload), 'r')).keys()))

    return res

In [122]:
result = load_from_file('mariadb')
print(len(result))

3


In [123]:
# calculate the capture differences
# the order of prints is always ycsb_a, ycsb_b, ycsb_c, cloudsuite_user&run, cloudsuite_run, cloudsuite_manual

superset = []
missing_syscalls = []
missing_syscalls_names = []
for j in range(0,len(result)):
    superset = np.union1d(superset, result[j])

for i in range(0,len(result)):
    dif = np.setdiff1d(superset, result[i])
    print('{} : {}'.format(len(dif),str(dif)))
    missing_syscalls.append(len(dif))
    missing_syscalls_names.append(dif)

0 : []
0 : []
1 : ['sendmmsg']


In [124]:
syscalls_with_groups = json.load(open('../resources/syscalls_with_groups.json', 'r'))
groups_with_syscalls = json.load(open('../resources/groups_with_syscalls.json', 'r'))

def groups(capture):
    g = []
    for syscall in capture:
        group_name = syscalls_with_groups[syscall]
        if not (group_name in g):
            g.append(group_name)
    return g

def syscalls(group_arr):
    syscall_arr = []
    for group in group_arr:
        for s in groups_with_syscalls[group]:
            syscall_arr.append(s)
    return syscall_arr

In [125]:
mined_syscalls = []
additional_syscalls = []

# calculate the capture differences with extrapolated syscalls from group
for i in range(0,len(result)):
    extrapolated = syscalls(groups(result[i]))

    print('{} traced calls'.format(len(result[i])))
    print('{} groups'.format(len(groups(result[i]))))
    additional_syscalls.append(len(extrapolated) - len(result[i]))
    print('{} additional Syscalls through extrapolation'.format(len(extrapolated) - len(result[i])))

    dif = np.setdiff1d(superset, extrapolated)
    print('{} : {}'.format(len(dif),str(dif)))


    print('Groups blocking additional Syscalls: {}'.format(groups(np.setdiff1d(missing_syscalls_names[i], dif))))
    mined_syscalls.append(missing_syscalls[i] - len(dif))
    print('-----------------')


103 traced calls
54 groups
110 additional Syscalls through extrapolation
0 : []
Groups blocking additional Syscalls: []
-----------------
103 traced calls
54 groups
110 additional Syscalls through extrapolation
0 : []
Groups blocking additional Syscalls: []
-----------------
102 traced calls
54 groups
111 additional Syscalls through extrapolation
0 : []
Groups blocking additional Syscalls: ['Socket Send']
-----------------


In [126]:
# Calculate Propability of hitting additional amount of mined syscalls through random selection
# as opposed to using groups

for i in range(0, len(result)):
    f = np.random.hypergeometric(ngood= missing_syscalls[i],
                                 nbad= 357 - len(result[i]) - missing_syscalls[i],
                                 nsample= additional_syscalls[i],
                                 size=10000000)

    print(sum(f >= mined_syscalls[i]) / 10000000.)



1.0
1.0
0.4353182
